<a href="https://colab.research.google.com/github/Usnish-Mukherjee/Intrusion_Detection/blob/master/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import numpy as np
from glob import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [106]:
nd1 = np.zeros((0,79),dtype=object)
data_set=pd.read_csv('Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv')
nd1=np.vstack((nd1,data_set))
data_set=pd.read_csv('Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv')
nd1=np.vstack((nd1,data_set))
data_set=pd.read_csv('Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv')
nd1=np.vstack((nd1,data_set))


In [107]:
nd1.shape

(1992746, 79)

In [108]:
Dt1=nd1[:,:-1].astype(float)

In [109]:
LNMV1=nd1[~np.isnan(Dt1).any(axis=1),-1]
DtNMV1=Dt1[~np.isnan(Dt1).any(axis=1)]
#Remove Inf values
LNMIV1=LNMV1[~np.isinf(DtNMV1).any(axis=1)]
DtNMIV1=DtNMV1[~np.isinf(DtNMV1).any(axis=1)]

In [110]:
del(DtNMV1)


In [111]:
DtNMIV1.shape

(1983658, 78)

In [112]:
MCDt1=DtNMIV1[LNMIV1 !='Benign',:]

In [113]:
MCDt1.shape

(848381, 78)

In [114]:
MCL1=LNMIV1[LNMIV1!='Benign']

In [115]:
LNMIV1.shape,MCL1.shape

((1983658,), (848381,))

In [116]:
np.save('NMCx1', MinMaxScaler().fit_transform(MCDt1))
np.save('NMCy1', LabelEncoder().fit_transform(MCL1))

In [117]:
import numpy as np
import os
import tensorflow as tf
import keras
from keras import models
from keras import layers

In [118]:
x=np.load('NMCx1.npy')
y=np.load('NMCy1.npy')
x=x.astype(np.float32)

In [119]:
x=np.delete(x,0,1)

In [120]:
(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[     0 686012]
 [     1   1730]
 [     2 160639]]


In [121]:
x.shape

(848381, 77)

In [122]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=123)

In [123]:
from keras.utils import to_categorical
ztrain = to_categorical(ytrain, num_classes=None)
ztest = to_categorical(ytest, num_classes=None)

In [124]:
ztrain.shape

(678704, 3)

In [125]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(256,input_dim=x.shape[1],activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(64,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(np.unique(y).shape[0],activation='softmax',use_bias=True)) 
  return model

from keras.optimizers import * 
model = create_model()
opt = Adam(learning_rate=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['categorical_accuracy'])
model.summary()
history=model.fit(xtrain , ztrain ,batch_size=256 , epochs=100 ,verbose=1 ,shuffle=True)
result = model.evaluate(xtest,ztest)
dict(zip(model.metrics_names, result))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 256)               19968     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)               

{'categorical_accuracy': 0.9999587535858154, 'loss': 0.0005346218822523952}

In [126]:
pred=model.predict(xtest)
y_pred = np.argmax(pred, axis=-1)

In [127]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, y_pred))
print("The precision score is",metrics.precision_score(ytest, y_pred,average='weighted'))
print("The recall score is",metrics.recall_score(ytest, y_pred,average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, y_pred,average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, y_pred))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, y_pred))
print('\n')

The accuracy score is 0.9999587451451876
The precision score is 0.9999587472487239
The recall score is 0.9999587451451876
The f1-score is 0.9999587434185853


[[137278      0      0]
 [     0    344      0]
 [     7      0  32048]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    137278
           1       1.00      1.00      1.00       344
           2       1.00      1.00      1.00     32055

    accuracy                           1.00    169677
   macro avg       1.00      1.00      1.00    169677
weighted avg       1.00      1.00      1.00    169677





In [128]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc_history=rfc.fit(xtrain,ytrain)
rfc_predict = rfc.predict(xtest)

In [129]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, rfc_predict.round()))
print("The precision score is",metrics.precision_score(ytest, rfc_predict.round(),average='weighted'))
print("The recall score is",metrics.recall_score(ytest, rfc_predict.round(),average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, rfc_predict.round(),average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, rfc_predict))
print('\n')

The accuracy score is 0.9997230031176884
The precision score is 0.9997233547119975
The recall score is 0.9997230031176884
The f1-score is 0.9997230742548636


[[137233      0     45]
 [     0    344      0]
 [     2      0  32053]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    137278
           1       1.00      1.00      1.00       344
           2       1.00      1.00      1.00     32055

    accuracy                           1.00    169677
   macro avg       1.00      1.00      1.00    169677
weighted avg       1.00      1.00      1.00    169677



